# Une courte introduction à Stan

https://mc-stan.org/

Stan est probablement le langage probabiliste le plus populaire, utilisé dans de nombreux domaines d'applications.

Pour ce TP, nous utiliserons 

- Jupyter (pour intéragir avec le notebook),
- L'interface Python [CmdStanPy](https://mc-stan.org/cmdstanpy/),
- Numpy (pour le calcul vectoriel),
- Pandas (pour manipuler les données et les resultats),
- Matplotlib et ipywidgets pour les visualisations.

In [ ]:
from cmdstanpy import CmdStanModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Un programme Stan contient plusieurs sections.
Les plus utilisées sont les suivantes

```stan
data {
  // input data
}
parameters {
  // latent random variables
}
model {
  // model definition
}
```

Il existe d'autres sections (`transformed data`, `transformed parameters`, `generated quantities`) qui permettent de faire du pre/post-traitement des données et des paramètres.

Par exemple, le programme suivant implémente le modèle classique qui infère le biais d'une pièce à partir d'observations statistiques.

In [ ]:
coin_code = """
data {
  int N; // number of observations
  int<lower=0, upper=1> y[N]; // array of observations
}
parameters {
  real<lower=0, upper=1> z; // coin bias
}
model {
  for (i in 1:N){
      y[i] ~ bernoulli(z);
  }
}
"""

# CmdStanPy can only read stan code from a file.
with open("./coin.stan", "w") as f:
    print(coin_code, file=f)

On peut ensuite instancier un modèle Stan à partir de ce code, puis lancer l'inférence sur des données concrètes.

In [ ]:
# Dict fields must coincide with model's data.
data = {
    'N' : 10,
    'y' : [0, 1, 1, 0, 0, 0, 0, 0, 0, 0] 
}

stan_model = CmdStanModel(stan_file="coin.stan") # instantiate the model
stan_fit = stan_model.sample(data, chains=1) # run inference (1 chains to limit cpu)
stan_fit.summary() # Print result summary

Il est possible de récupérer les échantillons de la distribution a posteriori à l'aide de la méthode `stan_variable`.  
Par exemple `stan_fit.stan_variable("z")` renvoie un tableau numpy dont la première dimension correspond au nombre d'échantillons (par défaut 1000).
Les dimensions suivantes correspondent aux dimensions du paramètre demandé (ici un simple flottant).

On peut vérifier que la moyenne des échantillons (le long de la première dimension) est bien celle affichée par la méthode `summary`.

In [ ]:
z_samples = stan_fit.stan_variable("z")
print(f"z samples shape: {z_samples.shape}")
print(f"z samples mean: {z_samples.mean(axis=0)}")

**Attention.**

- Les tableaux sont indicés à partir de 1 en Stan (mais 0 en Python)
- Tous les paramètres (variables déclarées dans `parameters`) doivent être continus.
- `~` ne peut être utilisé qu'avec des paramètres ou des données (variables aléatoires)
- Des variables intermédiaires peuvent être définis dans le modèle (e.g., `real foo; foo = 42;`) mais sans contrainte.

Le manuel Stan est disponible à l'adresse suivante : https://mc-stan.org/docs/2_28/stan-users-guide/index.html

## Exercice 1 : Baseball Hits 1970.

*Adapté du Tutorial ["Hierarchical Partial Pooling for Repeated Binary Trials"](https://mc-stan.org/users/documentation/case-studies/pool-binary-trials.html), B. Carpenter, 2016*

Dans cet exercice nous essayons de prevoir les performances des joueurs de baseball lors de la Major League Baseball season de 1970.  
Les données (Effron et Morris 1975) comprennent pour chaque batteur :
- Le nom, prénom, 
- Le nombre de succès après les 45 premiers essais,
- Le nombre d'essais restants jusqu'à la fin de la saison,
- Le nombre de succès restants jusqu'à la fin de la saison.

**Objectif.** Le but de ce problème est de prédire, pour chaque joueur, le nombre de succès restant à partir des resultats obtenus sur les 45 premiers essais.

In [ ]:
df = pd.read_csv("EfronMorrisBB.txt", sep="\t")
df = df.rename(columns={"At-Bats": "Bats", "RemainingAt-Bats": "RemainingBats"})
df["RemainingHits"] = df["SeasonHits"] - df["Hits"]
df = df[["FirstName", "LastName", "Bats", "Hits", "RemainingBats","RemainingHits"]]

# Dictionnary contaning all the data.
data = {
    "N": df.shape[0],
    "K": df["Bats"].to_numpy(),
    "y": df["Hits"].to_numpy(),
}

df

**Question 1.** On suppose dans un premier temps que tous les joueurs ont la même probabilité de réussite $\theta$ à chaque essai.  
Proposer un premier modèle qui permet d'inférer ce paramètre.

*Note.* voir loi binomiale.

In [ ]:
baseball1_code="""
data {
    int N;
    int<lower=0> K[N];
    int<lower=0> y[N];
}
parameters {
    // TODO
}
model {
    // TODO
}
"""

with open("./baseball1.stan", "w") as f:
    print(baseball1_code, file=f)

stan_model = CmdStanModel(stan_file="baseball1.stan")
stan_fit = stan_model.sample(data, chains=1)
stan_fit.summary()

**Question 2.** En déduire les predictions moyennes pour chacun des joueurs sachant le nombre d'essais restant jusqu'à la fin de la saison (colonne `df.RemainingBats`).

In [ ]:
df["Prediction1"] = None # TODO
df

Ce premier résultat est un bon début, mais il n'est pas très réaliste.
En pratique, les statistiques de certains batteurs sont [nettement supérieurs](https://www.baseball-reference.com/awards/hof_batting.shtml) à ceux des autres.

**Question 3.** Proposer un nouveau modèle, où les chances de succès de chaque joueur sont indépendantes des autres.

In [ ]:
baseball2_code="""
    // TODO
"""

with open("./baseball2.stan", "w") as f:
    print(baseball2_code, file=f)

stan_model = CmdStanModel(stan_file="baseball2.stan")
stan_fit = stan_model.sample(data, chains=1)
stan_fit.summary()

**Question 4.** En déduire les nouvelles predictions pour chacun des joueurs sachant le nombre d'essais restant jusqu'à la fin de la saison.

*Note.* On pourra utiliser `stan_fit.stan_variable("theta")`.
Comme précédemment, on prendra pour chaque joueur la valeur moyenne de theta.

In [ ]:
df["Prediction2"] = None # TODO
df

Pour finir, on voudrait ajouter des effets de population globaux à tous les joueurs.
Par exemple, si tous les autres joueurs font une saison exceptionnelle, il est probable que Berry aura lui aussi de très bons résultats.

On suppose maintenant que pour chaque joueur le paramètre $\theta$ suit une loi $\rm{Beta}(\alpha, \beta)$ (comme pour la pièce biaisée), où les paramètres $\alpha$, et $\beta$ sont communs à tous les joueurs.
On a maintenant un modèle hierarchique (le paramètre $\theta$ dépend lui même de 2 autres paramètres).

Il peut être difficile de trouver une bonne distribution a priori pour ces "hyperparamètres" $\alpha$, et $\beta$.
Pour contourner ce problème, on peut "reparamétriser" le modèle en introduisant les paramètres $\phi \in [0, 1]$ et $\kappa \in [1, \infty)$ tels que : 

$$
\alpha = \kappa * \phi
\quad
\beta = \kappa * (1 - \phi)
$$

La litérature propose alors les distributions a priori suivantes pour $\kappa$ et $\phi$ :
$$
\phi \sim \rm{Uniform}(0, 1)
\quad
\kappa \sim \rm{Pareto}(1, 1.5)
$$


**Question 5.** Implémenter ce dernier modèle en Stan.

In [ ]:
baseball3_code = """
    // TODO
"""

with open("./baseball3.stan", "w") as f:
    print(baseball3_code, file=f)

stan_model = CmdStanModel(stan_file="baseball3.stan")
stan_fit = stan_model.sample(data, chains=1)
stan_fit.summary()

**Question 6.** En déduire les nouvelles prédictions pour chaque joueur.

In [ ]:
df["Prediction3"] = None # TODO
df

## Exercice 2 : Variations sur une régression linéaire.

**Objectif.** Le but de cet exercice est d'utiliser Stan pour trouver une regression linéaire.  
Plutôt que de trouver la meilleure solution, on cherche ici une distribution de regressions possibles.

On considère les données bruitées suivantes.

In [ ]:
N = 8
noise = 0.25
lower = 0
upper = 10

x_obs = np.linspace(lower, upper, N)
y_obs = 2 * np.tanh(4 * (x_obs - upper) / upper) + noise * np.random.randn(N)

data = {
    'N': N,
    'low': lower,
    'up': upper,
    'x': x_obs,
    'y': y_obs
}

plt.scatter(x_obs, y_obs, color='red')

**Question 1.** Proposer un premier modèle de regression linéaire $y \sim \mathcal{N}(ax + b, \sigma)$.  
On cherche à estimer les paramètres $a$, $b$, et $\sigma$ (bruit).

In [ ]:
regression1_code = """
    // TODO
"""

with open("./regression1.stan", "w") as f:
    print(regression1_code, file=f)

stan_model = CmdStanModel(stan_file="regression1.stan")
stan_fit = stan_model.sample(data, chains=1)
stan_fit.summary()

On peut maintenant afficher quelque échantillons pour observer la distribution obtenue.

In [ ]:
for i in range(500):
    x = np.linspace(lower, upper, 2)
    a = stan_fit.stan_variable("a")[i]
    b = stan_fit.stan_variable("b")[i]
    plt.plot(x, a * x + b, color='blue', alpha=0.1, zorder=0)
    
plt.scatter(x_obs, y_obs, color='red', zorder=1)

Le résultat correspond à nos attente, mais n'est pas formidable pour notre jeux de données où on observe une rupture de pente.

**Question 2.** Proposer un nouveau modèle avec un paramètre supplémentaire `cut` pour ce point de rupture et deux regressions : avant et après ce point.
On pourra representer les paramètres `a` et `b` par des vecteurs à 2 dimensions.

In [ ]:
regression2_code = """
    // TODO
"""

with open("./regression2.stan", "w") as f:
    print(regression2_code, file=f)

stan_model = CmdStanModel(stan_file="regression2.stan")
stan_fit = stan_model.sample(data, chains=1)
stan_fit.summary()

On peut à nouveau visualiser la distribution obtenue.

In [ ]:
for i in range(500):
    a = stan_fit.stan_variable("a")[i]
    b = stan_fit.stan_variable("b")[i]
    cut = stan_fit.stan_variable("cut")[i]
    x = np.linspace(lower, cut, 2)
    plt.plot(x, a[0] * x + b[0], color='blue', alpha=0.1, zorder=0)
    x = np.linspace(cut, upper, 2)
    plt.plot(x, a[1] * x + b[1], color='blue', alpha=0.1, zorder=0)
    
plt.scatter(x_obs, y_obs, color='red', zorder=1)

**Question 3.** Proposer maintenant un modèle ou le nombre (maximal) de changement de pente $C$ est donné.  
(Pour simplifier on pourra fixer la valeur du bruit $\sigma$ dans ce modèle à une valeur raisonnable).
Les points de rupture `cuts` pourront être stockés dans un vecteur trié de dimension $C$ (cf. fonction `sort_asc`).

In [ ]:
regression3_code = """
    // TODO
"""

with open("./regression3.stan", "w") as f:
    print(regression3_code, file=f)

On peut maintenant essayer le même jeu de données avec plus de rupture de pente.

In [ ]:
C = 4

stan_model = CmdStanModel(stan_file="regression3.stan")
stan_fit = stan_model.sample(data = {**data, 'C':C}, chains=1)

In [ ]:
ax = plt.gca()
ax.set_ylim(-3, 3)
for i in range(100):
    cuts = np.append(stan_fit.stan_variable("cuts")[i], upper)
    a = stan_fit.stan_variable("a")[i]
    b = stan_fit.stan_variable("b")[i]
    low = lower
    for j in range(0,C+1):
        up = cuts[j]
        x = np.linspace(low, up, 2)
        plt.plot(x, a[j] * x + b[j], color='blue', alpha=0.1, zorder=0)
        low = up
    
plt.scatter(x_obs, y_obs, color='red', zorder=1)

Et maintenant sur un jeu de données plus compliqué.

In [ ]:
N = 20
x_obs = np.linspace(lower, upper, N)
y_obs = 2 * np.sin(x_obs)

data = {
    **data,
    'N': N, 
    'C': C,
    'x': x_obs,
    'y': y_obs
}

plt.scatter(x_obs, y_obs, color='red', zorder=1)

In [ ]:
stan_model = CmdStanModel(stan_file="regression3.stan")
stan_fit = stan_model.sample(data, chains=1)

In [ ]:
ax = plt.gca()
ax.set_ylim(-3, 3)
for i in range(100):
    cuts = np.append(stan_fit.stan_variable("cuts")[i], upper)
    a = stan_fit.stan_variable("a")[i]
    b = stan_fit.stan_variable("b")[i]
    low = lower
    for j in range(0,C+1):
        up = cuts[j]
        x = np.linspace(low, up, 2)
        plt.plot(x, a[j] * x + b[j], color='blue', alpha=0.1, zorder=0)
        low = up
    
plt.scatter(x_obs, y_obs, color='red', zorder=1)